🧩 Project: Smart Student Report System

1. Define a **Student** model using **Pydantic** for data validation.
2. Validate student data:
    - name (string)
    - marks (list of integers between 0 and 100)
    - email (valid email format)

3. Create a **from_string** class method that accepts input in the format: 'Name,email,mark1,mark2,...'
4. Write a **property** for the class that calculates the student's average mark.
5. Write a **static method** to convert the average mark to a grade:
    - A for average ≥ 90
    - B for average ≥ 75
    - C for average ≥ 65
    - F otherwise

In [ ]:
from pydantic import BaseModel, EmailStr
from typing import List

class Student(BaseModel):
    name: str
    marks: List[int]
    email: EmailStr

    @classmethod
    def check_marks(cls, marks):
        if not marks:
            raise ValueError("Marks cannot be empty")
        if any(m < 0 or m > 100 for m in marks):
            raise ValueError("Marks must be between 0 and 100")
        return marks

    @property
    def average(self) -> float:
        return sum(self.marks) / len(self.marks)

    @staticmethod
    def get_grade(average: float) -> str:
        if average >= 90: return "A"
        elif average >= 75: return "B"
        elif average >= 60: return "C"
        else: return "F"

    @classmethod
    def from_string(cls, data: str):
        """Create Student from string: 'Name,email,mark1,mark2,...'"""
        name, email, *marks = data.split(',')
        marks = list(map(int, marks))
        cls.check_marks(marks)
        return cls(name=name, email=email, marks=marks)


## Excersice :)

1. Write a decorator for logging:
    - Before the function call, print **"Running <function_name>"**.
    - After the function call, print **"Completed <function_name>"**.

2. Write a **StudentManager** class that maintains a list of Student objects.
3. Write an **add_student** method that adds a student to the list and uses the logging decorator.
4. Write a **show_summary** method that returns a dictionary mapping each student's name to their average score, and uses the logging decorator.
5. Write a **top_students** generator that yields students whose average score is greater than a specified threshold.

In [ ]:
from typing import List, Generator, Callable
import functools

# Decorator to log actions
def log_action(func: Callable):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print(f"Running {func.__name__}...")
        result = func(*args, **kwargs)
        print(f"Completed {func.__name__}\n")
        return result
    return wrapper

class StudentManager:
    def __init__(self):
        self.students: List[Student] = []

    @log_action
    def add_student(self, student: Student) -> None:
        self.students.append(student)

    @log_action
    def show_summary(self) -> None:
        summary = {s.name: Student.get_grade(s.average) for s in self.students}
        print("Student Grades:", summary)

    def top_students(self, threshold: float) -> Generator:
        for s in self.students:
            if s.average >= threshold:
                yield s

@log_action
def run_demo():
    mgr = StudentManager()

    s1 = Student.from_string("Alice,alice@example.com,90,95,88")
    s2 = Student.from_string("Bob,bob@example.com,70,65,75")
    s3 = Student.from_string("Charlie,charlie@example.com,85,80,90")
    # Test this:
    # s4 = Student.from_string("Charlie,charlie@example.com,120,80,-1")

    mgr.add_student(s1)
    mgr.add_student(s2)
    mgr.add_student(s3)
    # mgr.add_student(s3)

    mgr.show_summary()

    print("Top students (avg >= 80):")
    for top in mgr.top_students(80):
        print(f"{top.name} - Avg: {top.average:.2f}")



In [ ]:
run_demo()